In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
jdk_version = '11'
hadoop_version = 'hadoop2.7'

# Set Environment Variables
os.environ['SPARK_VERSION'] = spark_version
os.environ['JDK_VERSION'] = jdk_version
os.environ['HADOOP_VERSION'] = hadoop_version
os.environ['JAVA_HOME'] = f'/usr/lib/jvm/java-{jdk_version}-openjdk-amd64'
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-{hadoop_version}'

# Update modules and install Spark and Java
!apt-get update
!apt-get install openjdk-$JDK_VERSION-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Level-Two-Starter-Code").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
raw_data_file = "amazon_reviews_us_Video_Games_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + raw_data_file
spark.sparkContext.addFile(url)
# Read and show the data
# Read and show the data
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

# Filter by Votes

In [8]:
# Filter for greater than 20 total votes
from pyspark.sql.functions import col
cleaned_df = df.filter(col("total_votes")  >=20)
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [9]:
# Filter for greater than 50% helpful vote percentage
cleaned_df = df.filter(col("helpful_votes")  >=50)
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34170274|R127WEQY2FM1T3|B00W435BU0|     487294121|       Madden NFL 16|     Video Games|          1|          147|        175|   N|                Y|Hot Mess. Don't Buy.|This is an absolu...| 2015-08-31|
|         US|   37932477|R2FJ94555FZH32|B00WJ0Z194|     116945357|Infinity 3.0 Star...|     Video Games|          2|    

# Describe Stats

In [13]:
# Paid reviews
from pyspark.sql.functions import col, avg
paid_df = cleaned_df.filter(cleaned_df['vine']== 'Y')
paid_df.describe().show(5)

+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|product_id|     product_parent|       product_title|product_category|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+-----------+
|  count|         27|                  27|            27|        27|                 27|                  27|              27|                27|                27|               27|  27|               27|               

In [14]:
# Unpaid reviews
unpaid_df = cleaned_df.filter(cleaned_df['vine']== 'N')
unpaid_df.describe().show(5)

+-------+-----------+--------------------+--------------+------------------+-------------------+--------------------+----------------+-----------------+------------------+------------------+----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|        product_id|     product_parent|       product_title|product_category|      star_rating|     helpful_votes|       total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+------------------+-------------------+--------------------+----------------+-----------------+------------------+------------------+----+-----------------+--------------------+--------------------+-----------+
|  count|       8734|                8734|          8734|              8734|               8734|                8734|            8734|             8734|              8734|              8734|8734| 

# Determine the percentage of five-star review among vine reviews

In [15]:
# CODE HERE
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(f'Number of paid reviews {paid_number}')
print(f'Number of paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')

Number of paid reviews 27
Number of paid five star reviews 13
Percantage of paid reviews that are five stars 48.148148148148145%


# Determine the percentage of five-star review among non-Vine reviews

In [16]:
# CODE HERE
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)

print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews 8734
Number of unpaid five star reviews 3942
Percantage of paid reviews that are five stars 45.13395923975269%
